In [ ]:
from library.analysis import loadDataset, testSets
from library.generators.XConvGeN import XConvGeN, GeneratorConfig
from library.timing import timing
from fdc.fdc import FDC
import numpy as np
import tensorflow as tf

In [ ]:
data = loadDataset(testSets[0])

In [ ]:
print((len(data.data), len(data.data0), len(data.data1)))

In [ ]:
fdc = FDC()
fdc.nom_list = [0]
fdc.cont_list = list(range(data.data0.shape[1]))[1:]

In [ ]:
config = GeneratorConfig(n_feat=data.data0.shape[1], neb=5, genAddNoise=True, alpha_clip=0.1)
gen = XConvGeN(config=config, fdc=fdc, debug=True)

In [ ]:
gen.reset(data.data)

In [ ]:
t = timing("train")
t.start()
gen.train(data.data)
t.stop()
print(t)

# train: #1 225.9655s

In [ ]:
#train: #1 1634.8339s

syntheticPoints = gen.generateData(data.data1.shape[0])

In [ ]:
syntheticPoints[:5]

In [ ]:
data.data1[:5]

In [ ]:
import math
dists = [min([math.sqrt(sum(y*y)) for y in (data.data1 - p) ]) for p in syntheticPoints]
print( min(dists), sum(dists) / len(dists), max(dists) )

In [ ]:
v = gen.predictReal(data.data1)
(min(v), max(v), sum(v) / len(v))